In [24]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from torch.utils.data import Dataset
import pickle
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split


In [2]:
class ImageDataset(Dataset):
    
    def __init__(self, csv_files):
        super().__init__()
        
        with open(csv_files[0], 'rb') as f:
            X = np.array(pickle.load(f),dtype=np.float32)
             
        with open(csv_files[1], 'rb') as g:
            y = np.array(pickle.load(g))
        
        
        le=LabelEncoder()
        y_label=le.fit_transform(y)
        self.X= X.reshape(-1,1,28,28)
        self.y=y_label
        self.le =le
                
        
                
    def get_class_count(self):
        return len(self.le.classes_)
    
    def get_input_dim(self):
        return self.X.shape[1]
    
    def get_class_labels(self,y_label):
        le=self.le
        return le.inverse_transform(y_label)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx],self.y[idx]

In [26]:
im = ImageDataset(['train_image.pkl','train_label.pkl'])
input_dim= im.get_input_dim()
output_dim = im.get_class_count()

train_size = int(0.8 * len(im))
test_size = len(im) - train_size
train_dataset, test_dataset = random_split(im, [train_size, test_size])
train_loader= DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size, shuffle=False)

In [27]:
import torch.nn as nn
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [33]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.outer = nn.Linear(in_features=60, out_features=4)
        
    def forward(self,x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.outer(out)
        return out


In [34]:
model = Network()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as tensors with gradient accumulation abilities
        images = images.requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to tensors with gradient accumulation abilities
                images = images.requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.30120134353637695. Accuracy: 80
Iteration: 1000. Loss: 0.23643282055854797. Accuracy: 79
Iteration: 1500. Loss: 0.15540777146816254. Accuracy: 78
Iteration: 2000. Loss: 0.15448611974716187. Accuracy: 79
Iteration: 2500. Loss: 0.21673019230365753. Accuracy: 80
